In [2]:
import nengo
%load_ext nengo.ipynb
from nengo import spa

from collections import OrderedDict

D = 32
vocab = spa.Vocabulary(D, unitary=["ONE"])
number_dict = {"ONE":1, "TWO":2, "THREE":3, "FOUR":4, "FIVE":5,
               "SIX":6, "SEVEN":7, "EIGHT":8, "NINE":9}
number_ordered = OrderedDict(sorted(number_dict.items(), key=lambda t: t[1]))

number_range = 4
vocab.parse("NONE")
vocab.parse("ONE")
number_list = number_ordered.keys()
for i in range(number_range):
    vocab.add(number_list[i+1], vocab.parse("%s*ONE" % number_list[i]))

join_num = "+".join(number_list[0:number_range])
num_ord_filt = OrderedDict(number_ordered.items()[:number_range])

model = spa.SPA(vocab)

The nengo.ipynb extension is already loaded. To reload it, use:
  %reload_ext nengo.ipynb
ONE+TWO+THREE+FOUR


In [3]:
with model:
    model.vision = spa.State(D)
    model.motor = spa.State(D)
    model.mem = spa.State(D, feedback=1)
    
    actions = spa.Actions(
        "dot(vision, START) --> mem=ONE",
        "dot(vision, PLUS) --> mem=mem*ONE",
        "dot(vision, MINUS) --> mem=mem*~ONE",
        "dot(vision, DONE) --> motor=mem"
    )
    
    model.bg = spa.BasalGanglia(actions)
    model.thal = spa.Thalamus(model.bg)